In [1]:
%reset -f

In [2]:
from neural_bandits.modules.neural_ts_module import NeuralTSBanditModule
from neural_bandits.datasets.mnist import MNISTDataset

import lightning as pl
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.data import Subset
import random

In [3]:
class Network(nn.Module):
    def __init__(self, dim, hidden_size=100):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(dim, hidden_size)
        self.activate = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
    def forward(self, x):
        return self.fc2(self.activate(self.fc1(x)))

In [4]:
dataset = MNISTDataset()
print(dataset.context_size)
print(len(dataset))

network = Network(dataset.context_size * dataset.num_actions, hidden_size=100)

n_samples = 15000
indices = list(range(len(dataset)))
random.shuffle(indices)
subset_indices = indices[:n_samples]
mini_dataset = Subset(dataset, subset_indices)
print("Subset size:", len(mini_dataset))

train_loader = DataLoader(mini_dataset, batch_size=1, shuffle=True)
model = NeuralTSBanditModule(
        n_features = dataset.context_size * dataset.num_actions,
        network=network,
        early_stop_threshold=1e-3,
        num_grad_steps=1000,
        lambda_=1e-3,
        nu=1e-5,
        learning_rate=1e-3,
        train_freq=100,
        initial_train_steps=2000,
        max_grad_norm=5.0,
)

logger = pl.pytorch.loggers.CSVLogger("logs", name="neural_ucb", flush_logs_every_n_steps=100)
trainer = pl.Trainer(max_epochs=1, logger=logger, log_every_n_steps=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


784
70000
Subset size: 15000


In [5]:
trainer.fit(model, train_loader)


  | Name   | Type           | Params | Mode 
--------------------------------------------------
0 | bandit | NeuralTSBandit | 784 K  | train
--------------------------------------------------
784 K     Trainable params
0         Non-trainable params
784 K     Total params
3.137     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode
c:\Users\Parisa\anaconda3\envs\neural_bandits\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 15000/15000 [2:45:57<00:00,  1.51it/s, v_num=108, loss=0.0263, reward=1.000, regret=0.000, average_regret=0.104]   

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 15000/15000 [2:45:57<00:00,  1.51it/s, v_num=108, loss=0.0263, reward=1.000, regret=0.000, average_regret=0.104]


In [ ]:
# load metrics from the logger and plot
import pandas as pd
import numpy as np

metrics_csv = logger.log_dir + "/metrics.csv"
print(metrics_csv)
metrics = pd.read_csv(metrics_csv)
cumulative_reward = np.cumsum(metrics["reward"][:15000])
cumulative_regret = np.cumsum(metrics["regret"][:15000])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(cumulative_reward, label='Cumulative Reward')
plt.plot(cumulative_regret, label='Cumulative Regret')
plt.xlabel("steps")
plt.ylabel("cumulative reward/regret")
plt.legend()
plt.show()

In [ ]:
# average reward
print(sum(metrics["reward"][:10]) / 10)
print(sum(metrics["reward"][:1000]) / 1000)
print(sum(metrics["reward"][:15000]) / 15000)

print(sum(metrics["regret"][:10]) / 10)
print(sum(metrics["regret"][:1000]) / 1000)
print(sum(metrics["regret"][:15000]) / 15000)

In [ ]:
plt.plot(metrics["loss"][:15000], label='Loss')
plt.xlabel("steps")
plt.ylabel("loss")
plt.legend()
plt.show()